# Data cleaning 

### Hi! My name is Gustavo Montenegro, I hope this notebook can be enough easy to read for you.  

In this notebook I use the most popular neighborhoods in the boroughs which I want to compare. 
The Data Frame for each borough is in a CSV file that I´ve created in diferent notebooks. 

You can see in my github repository how I builded the Data Frames.
https://github.com/GustavoMontenegroVargas/Coursera_Capstone
- The 'C' Notebook has the Data from the Cuauhtémoc borough.The Downtown is there. 
- The 'T' Notebook has the Data from the Tláhuac borough. There is my local economy. 

I use these Data Frames in order to get two parameters: Rating and number of similar venues. With this parameters I will make an analysis. I want to answer this question: 'How can I open a similar venue in my local economy like the most popular from the Downtown?'

I know, If I want to open a venue, I need a Market study.  But with this idea someones can get a few useful information about their local economy.  

In this notebook I use the Foursquare API in order to get the recommended venues and the details of each one. You can see my complete code in order to extract information from a JSON file.

So, I didn´t use any of Machine learning algorithm, in fact, a good idea could be a multiple linear regressión with the historical data for each cluster of venues. We could predict the business performance. 

How I didn't have enough time to search Data Frames with that kind of information, then the parameters are the most common but they are very useful.

Note: When I´m saying 'The most popular neighborhoods in each borough' I want to say the neighborhoods that I've visited because they are very popular.


### The notebook starts here. 

I need to import some libraries.

In [44]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

With this code I can open the CSv that I have in the IBM cloud storage. This is the Data Frame of the Cuahtémoc borough. 

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_30e6154c81ce458e9a81cf92d1db1713 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='U8RMA1wSB4yc7S4Krymjrj-RqKmA_h_LspLsjEv_jbJo',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_30e6154c81ce458e9a81cf92d1db1713.get_object(Bucket='courseracapstone-donotdelete-pr-tjqo81z1sickby',Key='C.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

C = pd.read_csv(body)
C.head()


,Neighborhood,Postal Code,Latitude,Longitude
0,Algarin,6880,19.405455,-99.140086
1,Ampliación Asturias,6890,19.407207,-99.131016
2,Asturias,6850,19.404581,-99.130911
3,Atlampa,6450,19.456785,-99.156875
4,Buenavista,6350,19.446167,-99.152696


This is the Data Frame of the Tláhuac borough. 

In [3]:

body = client_30e6154c81ce458e9a81cf92d1db1713.get_object(Bucket='courseracapstone-donotdelete-pr-tjqo81z1sickby',Key='T.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

T = pd.read_csv(body)
T.head()


,Neighborhood,Postal Code,Latitude,Longitude
0,Agrícola Metropolitana,13280,19.290832,-99.052493
1,Ampliación José López Portillo,13419,19.313111,-99.027129
2,Ampliación La Conchita,13545,19.335036,-98.977113
3,Ampliación Los Olivos,13219,19.320556,-99.151701
4,Ampliación Santa Catarina,13120,19.318017,-98.963729


I need to get the most popular neigborhoods in a Data Frame from the Cuahtémoc borough. I wil call it 'CN'.

In [4]:
CN = C[4:5].append(C[6:14]).append(C[15:16]).append(C[22:24]).append(C[31:33]).append(C[36:37])
CN.reset_index(inplace = True)
CN.drop('index',axis=1,inplace=True)
CN

,Neighborhood,Postal Code,Latitude,Longitude
0,Buenavista,6350,19.446167,-99.152696
1,Centro (Área 1),6000,19.432700,-99.137600
2,Centro (Área 2),6010,19.439600,-99.136600
3,Centro (Área 3),6020,19.438400,-99.128500
4,Centro (Área 4),6040,19.430900,-99.149900
5,Centro (Área 5),6050,19.433700,-99.144300
6,Centro (Área 6),6060,19.431700,-99.129500
7,Centro (Área 7),6070,19.428900,-99.145200
8,Centro (Área 8),6080,19.426000,-99.138800
9,Condesa,6140,19.414864,-99.176429


In order to use the Foursquare API I need to define the credetials of my Coursera Capstone App. 

In [5]:
CLIENT_ID = 'A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU' # your Foursquare ID
CLIENT_SECRET = 'S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU
CLIENT_SECRET:S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV


When I make a call to Foursquere API I need this kind of code. Each call give me a JSON file. The JSON file is a dictonarie, so I can get the information with the keys. 
In this 'for loop' I will get the relevant data of each ten venues each neighborhood from the JSON files. How the 'endpoint' is 'explore', then Foursquere will give me the ten most popular venues by rating for each neighborhood. 

In [6]:
radius = 500 # This the raduis from the center of the neighborhood.
LIMIT = 10  # This is the number of venues that I want to get. 
ID = []
Name = []
#Rating = []

for j in range(len(CN)):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, CN['Latitude'][j],CN['Longitude'][j], VERSION, radius, LIMIT)
    print(CN['Neighborhood'][j], url , j)
    results = requests.get(url).json()
    for i in range(10):
        ID.append(results['response']['groups'][0]['items'][i]['venue']['id'])# I can extract the venue ID and put it in a list.
        Name.append(results['response']['groups'][0]['items'][i]['venue']['name'])# I can extract the Venue name and put it in a list.
        
print(len(ID))
len((Name))
        
    
    
    
   

Buenavista https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.446167000000003,-99.15269609999999&v=20180605&radius=500&limit=10 0
Centro (Área 1) https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.4327,-99.1376&v=20180605&radius=500&limit=10 1
Centro (Área 2) https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.4396,-99.1366&v=20180605&radius=500&limit=10 2
Centro (Área 3) https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.4384,-99.1285&v=20180605&radius=500&limit=10 3
Centro (Área 4) https://api.foursquare.c

150

How I got the lists of each feature that I want, I need to put them in a Data Frame. I will call it 'CN_Venues'.

In [7]:
CN_Venues = pd.DataFrame({"ID" : ID , "Name":Name }) # I can create a new Data Frame with the information from the lists. 
CN_Venues.head()

,ID,Name
0,5227a6a311d2e49199d6c6b8,Micro Teatro Mexico
1,4cd4a81567adf04d75ae2891,Quality Bakery
2,52c1d7ef498e4be2b5f785bd,Turin
3,505513b9e4b0b914257ee5f2,Postres Buenavista
4,4d10192759606dcb4531733e,Carl's Jr.


In [8]:
CN_Venues.shape

(150, 2)

So, I need to complete my 'CN_Venues' and get the rating feature. I don´t need the location features for each venue by now.  

In [9]:
M = len(CN_Venues) #define the number of rows in the Data Frame to build a for loop. 

This API call is diferent from the last one. This kind of call is premium. With my account I only have 500. Like the JSON file of recommended venues, I will get a new JSON file with the 'details' endpoint. This the same idea. I need to extract the relevant information. With this kind of call I can get the rating, stats and some other important features. But now, I want the rating feature. 

In [10]:
Rating = []
for k in range(M):
    
    venue_id = '{}'.format(CN_Venues['ID'][k]) # ID 
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    url
    result = requests.get(url).json()
    
    Rating.append(result['response']['venue']['rating'])#This line extract the rating feature from the JSON file.
    
len(Rating)

150

Now, I add the rating column to 'CN_Venues' Data Frame. 

In [11]:
CN_Venues['Rating'] = Rating # It is a easy way to add a new column from a list. 
CN_Venues.head()

,ID,Name,Rating
0,5227a6a311d2e49199d6c6b8,Micro Teatro Mexico,9.0
1,4cd4a81567adf04d75ae2891,Quality Bakery,8.8
2,52c1d7ef498e4be2b5f785bd,Turin,8.9
3,505513b9e4b0b914257ee5f2,Postres Buenavista,7.8
4,4d10192759606dcb4531733e,Carl's Jr.,7.8


In [12]:
CN_Venues.describe() # This the main information of the Rating column. 

,Rating
count,150.000000
mean,8.509333
std,0.379216
min,7.600000
25%,8.300000
50%,8.500000
75%,8.800000
max,9.400000


The 'explore' endpoint make a regular call. I need to get the main categorie of each venue. 

In [13]:
Categories = []
for j in range(len(CN)):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, CN['Latitude'][j],CN['Longitude'][j], VERSION, radius, LIMIT)
    print(CN['Neighborhood'][j], url , j)
    results = requests.get(url).json()
    for i in range(10):
        Categories.append(results['response']['groups'][0]['items'][i]['venue']['categories'][0]['name'])
        
print(len(Categories))

        
    

Buenavista https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.446167000000003,-99.15269609999999&v=20180605&radius=500&limit=10 0
Centro (Área 1) https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.4327,-99.1376&v=20180605&radius=500&limit=10 1
Centro (Área 2) https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.4396,-99.1366&v=20180605&radius=500&limit=10 2
Centro (Área 3) https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.4384,-99.1285&v=20180605&radius=500&limit=10 3
Centro (Área 4) https://api.foursquare.c

In [14]:
CN_Venues['Main Categorie'] = Categories # Add the new columns from a list. 
CN_Venues.head()

,ID,Name,Rating,Main Categorie
0,5227a6a311d2e49199d6c6b8,Micro Teatro Mexico,9.0,Public Art
1,4cd4a81567adf04d75ae2891,Quality Bakery,8.8,Bakery
2,52c1d7ef498e4be2b5f785bd,Turin,8.9,Candy Store
3,505513b9e4b0b914257ee5f2,Postres Buenavista,7.8,Dessert Shop
4,4d10192759606dcb4531733e,Carl's Jr.,7.8,Fast Food Restaurant


With this kind of code you can save your CSV from a notebook. Only you need yout notebook token. 

In [15]:
CNV_byRating = CN_Venues.sort_values('Rating',ascending = False) # With this line of code yo can put in order the Data Frame rows by a column.

In [16]:
CNV_byRating.reset_index(inplace = True)# You can reset the index with the new order. 
CNV_byRating.head(20)

,index,ID,Name,Rating,Main Categorie
0,90,4baec3d5f964a52018d63be3,Nevería Roxy,9.4,Ice Cream Shop
1,50,4b684e6ef964a520d0702be3,Museo de Arte Popular,9.4,Art Museum
2,117,4b819a4cf964a520fcb130e3,Fondo de Cultura Económica Rosario Castellanos,9.3,Bookstore
3,55,50ca02bf245f2d4aa8c2aeef,El Cardenal,9.3,Mexican Restaurant
4,10,55f476fe498e26410387a491,Hotel Historico Central,9.1,Hotel
5,120,56d76bac498ecdce6449901f,Tierra Garat,9.1,Cafeteria
6,80,4d49c47b11a36ea827282b1c,Zéfiro,9.1,Mexican Restaurant
7,140,4ccaca619b9346889cb65041,Kiosco Morisco,9.1,Plaza
8,35,4bfc1626d6f2c9b669b24ec8,Antiguo Colegio de San Ildefonso,9.0,Art Museum
9,102,4da0921c31a6b60c1a467618,Tout Chocolat,9.0,Dessert Shop


In [18]:

CNV_byRating.drop('index',axis=1, inplace = True)#You can drop the column that you don´t need anymore. 

This is the new Data Frame. I called it 'CNV_byRating'. I only show the first eighteen rows. In this slice are the ratings between 9 to 10. 

In [19]:
CNV_byRating.head(18) 

,ID,Name,Rating,Main Categorie
0,4baec3d5f964a52018d63be3,Nevería Roxy,9.4,Ice Cream Shop
1,4b684e6ef964a520d0702be3,Museo de Arte Popular,9.4,Art Museum
2,4b819a4cf964a520fcb130e3,Fondo de Cultura Económica Rosario Castellanos,9.3,Bookstore
3,50ca02bf245f2d4aa8c2aeef,El Cardenal,9.3,Mexican Restaurant
4,55f476fe498e26410387a491,Hotel Historico Central,9.1,Hotel
5,56d76bac498ecdce6449901f,Tierra Garat,9.1,Cafeteria
6,4d49c47b11a36ea827282b1c,Zéfiro,9.1,Mexican Restaurant
7,4ccaca619b9346889cb65041,Kiosco Morisco,9.1,Plaza
8,4bfc1626d6f2c9b669b24ec8,Antiguo Colegio de San Ildefonso,9.0,Art Museum
9,4da0921c31a6b60c1a467618,Tout Chocolat,9.0,Dessert Shop


Here is the 'T' Data Frame again.

In [20]:
T

,Neighborhood,Postal Code,Latitude,Longitude
0,Agrícola Metropolitana,13280,19.290832,-99.052493
1,Ampliación José López Portillo,13419,19.313111,-99.027129
2,Ampliación La Conchita,13545,19.335036,-98.977113
3,Ampliación Los Olivos,13219,19.320556,-99.151701
4,Ampliación Santa Catarina,13120,19.318017,-98.963729
5,Ampliación Selene,13430,19.284367,-98.991384
6,Ampliación Zapotitla,13315,19.335036,-98.977113
7,Cañada,13110,19.335036,-98.977113
8,Chichilaula,13123,19.335036,-98.977113
9,Del Mar,13270,19.320556,-99.151701


I need to get the most popular neighborhoods in the Tláhuac borough.  I will call it 'TN'. 

In [21]:
TN = T[0:1].append(T[9:10]).append(T[31:32]).append(T[34:35]).append(T[37:38]).append(T[61:66]).append(T[78:79])#With the append sentence you can creat a Data Frame by rows instead the columns. 
TN.reset_index(inplace=True)


In [22]:
TN.drop('index',axis=1,inplace=True)# You can drop the column that you don´t need anymore. 
TN

,Neighborhood,Postal Code,Latitude,Longitude
0,Agrícola Metropolitana,13280,19.290832,-99.052493
1,Del Mar,13270,19.320556,-99.151701
2,La Nopalera,13220,19.298380,-99.052562
3,La Turba,13250,19.320556,-99.151701
4,Los Olivos,13210,19.320556,-99.151701
5,Santa Ana,13060,19.267541,-99.008578
6,Santa Ana Centro,13300,19.301105,-99.036331
7,Santa Ana Norte,13300,19.301105,-99.036331
8,Santa Ana Poniente,13300,19.301105,-99.036331
9,Santa Ana Sur,13360,19.284663,-99.030658


In [23]:
TN.drop([1],axis=0,inplace=True)
TN.reset_index(inplace=True)
TN

,index,Neighborhood,Postal Code,Latitude,Longitude
0,0,Agrícola Metropolitana,13280,19.290832,-99.052493
1,2,La Nopalera,13220,19.298380,-99.052562
2,3,La Turba,13250,19.320556,-99.151701
3,4,Los Olivos,13210,19.320556,-99.151701
4,5,Santa Ana,13060,19.267541,-99.008578
5,6,Santa Ana Centro,13300,19.301105,-99.036331
6,7,Santa Ana Norte,13300,19.301105,-99.036331
7,8,Santa Ana Poniente,13300,19.301105,-99.036331
8,9,Santa Ana Sur,13360,19.284663,-99.030658
9,10,Zapotitla,13310,19.306475,-99.041657


In [24]:
TN.drop('index',axis=1,inplace=True)
TN

,Neighborhood,Postal Code,Latitude,Longitude
0,Agrícola Metropolitana,13280,19.290832,-99.052493
1,La Nopalera,13220,19.298380,-99.052562
2,La Turba,13250,19.320556,-99.151701
3,Los Olivos,13210,19.320556,-99.151701
4,Santa Ana,13060,19.267541,-99.008578
5,Santa Ana Centro,13300,19.301105,-99.036331
6,Santa Ana Norte,13300,19.301105,-99.036331
7,Santa Ana Poniente,13300,19.301105,-99.036331
8,Santa Ana Sur,13360,19.284663,-99.030658
9,Zapotitla,13310,19.306475,-99.041657


I will use the 'explore' endpoint to get almost the same features like in the CN Data Frame. 
The features will be 'Name', 'Categorie' and 'Venue_ID'. I need the Venue_ID in order to use the 'detail' endpoint a get some other features. Now, my radius is more bigger than the CN Data Frame because this is a lower economy than the Cuahtémoc borough. Maybe I wiil get a duplicate information.

In [25]:
radius = 4000
LIMIT = 50
T_ID = []
T_Name = []
T_Categories = []


for j in range(len(TN)):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, TN['Latitude'][j],TN['Longitude'][j], VERSION, radius, LIMIT)
    print(TN['Neighborhood'][j], url , j)
    results = requests.get(url).json()
    for i in range(50):
        T_ID.append(results['response']['groups'][0]['items'][i]['venue']['id'])
        T_Name.append(results['response']['groups'][0]['items'][i]['venue']['name'])
        T_Categories.append(results['response']['groups'][0]['items'][i]['venue']['categories'][0]['name'])
                        
        
print(len(T_ID))
print(len(T_Name))
print(len(T_Categories))


Agrícola Metropolitana https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.2908319,-99.0524934&v=20180605&radius=4000&limit=50 0
La Nopalera https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.2983799,-99.05256225&v=20180605&radius=4000&limit=50 1
La Turba https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.320556250000006,-99.15170107077653&v=20180605&radius=4000&limit=50 2
Los Olivos https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.320556250000006,-99.15170107077653&v=20180605&radius=4000&limit=50 3
Santa 

I need a new Data Frame with the new Features. I will call it ' TN_Venues'.  

In [26]:
TN_Venues = pd.DataFrame({'ID':T_ID,'Name':T_Name,'Main Categorie':T_Categories}) # You can create a new Data Frame from the lists. 
TN_Venues.head()

,ID,Name,Main Categorie
0,4cae778aeb65b1f760595ccd,Los Taquitos,Taco Place
1,4eaadaf4b803cf1ffb47a0d8,Carnitas La Flor de Cotija,Mexican Restaurant
2,50521cfbe4b02dee62ca3635,Carnitas Estilo Michoacán El Paisa,Taco Place
3,4f20c6d4e4b08ed6b4ca47a2,Don Cuco,Taco Place
4,4f6fe82ee4b07e41f09878a5,Sushisimo,Sushi Restaurant


I need the latitude and longitude Features. I will get them with the 'explore' endpoint. 

In [27]:
radius = 4000
LIMIT = 50
lat = [] #Lat
lon = [] #Lon



for j in range(len(TN)):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, TN['Latitude'][j],TN['Longitude'][j], VERSION, radius, LIMIT)
    print(TN['Neighborhood'][j], url , j)
    results = requests.get(url).json()
    for i in range(50):
        lat.append(results['response']['groups'][0]['items'][i]['venue']['location']['lat'])#You can extract the latitude feature.
        lon.append(results['response']['groups'][0]['items'][i]['venue']['location']['lng'])#You can extract the longitude feature.
                 
print(len(lat))
print(len(lon))

Agrícola Metropolitana https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.2908319,-99.0524934&v=20180605&radius=4000&limit=50 0
La Nopalera https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.2983799,-99.05256225&v=20180605&radius=4000&limit=50 1
La Turba https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.320556250000006,-99.15170107077653&v=20180605&radius=4000&limit=50 2
Los Olivos https://api.foursquare.com/v2/venues/explore?client_id=A35JKB4OB3YP5S1IY45IN0TRVEQ34RKQHPYUM5OHP354WPHU&client_secret=S513RXSSCT3GZDABQ4BSJJVULB2U0W23OU1BQTT0I2F1PLTV&ll=19.320556250000006,-99.15170107077653&v=20180605&radius=4000&limit=50 3
Santa 

In [28]:
TN_Venues['Latitude'] = lat # I add the new column from the Latitude list
TN_Venues['Longitude'] = lon # I add the new column from the longitude List
TN_Venues.head()

,ID,Name,Main Categorie,Latitude,Longitude
0,4cae778aeb65b1f760595ccd,Los Taquitos,Taco Place,19.295809,-99.055430
1,4eaadaf4b803cf1ffb47a0d8,Carnitas La Flor de Cotija,Mexican Restaurant,19.294897,-99.057706
2,50521cfbe4b02dee62ca3635,Carnitas Estilo Michoacán El Paisa,Taco Place,19.287660,-99.050957
3,4f20c6d4e4b08ed6b4ca47a2,Don Cuco,Taco Place,19.291856,-99.055175
4,4f6fe82ee4b07e41f09878a5,Sushisimo,Sushi Restaurant,19.297009,-99.054334


Now, I know that maybe I could get duplicate values. So, lets check. 

In [29]:
print(len(TN_Venues['Name'].unique()))# With the unique() function you can determinate the non-duplicate values.
sorted(TN_Venues['Name'].unique())


155


['"El Dique"',
 'Alberca - Asturiano',
 'Alitas "Las Carbonas"',
 'Antojitos Don Rafa',
 'Asadero Charolai',
 'Barbacoa 2 hermanos',
 'Barbacoa El Compadre',
 'Barbacoa Sta Cecilia',
 'Be Grand Coyoacán',
 'Beverly Gym',
 'Birria De Chivo "La Huacana"',
 'Bosque de Tláhuac',
 'CENTRO VETERINARIO PAWS',
 'Caballo Azteca',
 'Café Punta del Cielo',
 'Caldos de Gallina "El Pollito"',
 'Campos de Futbol Tempiluli',
 'Cancha Molinon',
 'Cantera PUMAS',
 'Carnitas Don Gaby',
 'Carnitas El Cuñadito',
 'Carnitas Estilo Michoacán El Paisa',
 'Carnitas La Flor de Cotija',
 'Carnitas Y Barbacoa "Hermanos Rodriguez"',
 'Casa del abuelo Cafeteria',
 'Caution Gym',
 'Cecina De Yecapixtla',
 'Centro Asturiano Canchas De Tenis',
 'Centro Asturiano de México, A. C.',
 'Centro Cultural Zapotitlán',
 'Cervecentro Estación 19',
 'Chocko Churro',
 'Cielito Querido Cafe',
 'Crazy Sushi',
 'Don Cuco',
 'El PORTAL de la CECINA',
 'El Pez de Oro',
 'El Pretexto (Café El Pretexto)',
 'El Rincón de Peribán',
 'El

In [30]:
print(len(TN_Venues['Main Categorie'].unique()))
sorted(TN_Venues['Main Categorie'].unique())

58


['American Restaurant',
 'Art Museum',
 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Bar',
 'Beer Garden',
 'Breakfast Spot',
 'Brewery',
 'Burger Joint',
 'Café',
 'Clothing Store',
 'Coffee Shop',
 'College Gym',
 'Convenience Store',
 'Donut Shop',
 'Fair',
 'Food Stand',
 'Food Truck',
 'Football Stadium',
 'Garden',
 'General Entertainment',
 'Gym',
 'Gym / Fitness Center',
 'Hobby Shop',
 'Ice Cream Shop',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Lounge',
 'Market',
 'Mexican Restaurant',
 'Music Venue',
 'Night Market',
 'Other Great Outdoors',
 'Park',
 'Performing Arts Venue',
 'Pet Store',
 'Pharmacy',
 'Pizza Place',
 'Plaza',
 'Pool',
 'Residential Building (Apartment / Condo)',
 'Restaurant',
 'Salad Place',
 'Seafood Restaurant',
 'Soccer Field',
 'Soccer Stadium',
 'Sports Club',
 'Stadium',
 'Stationery Store',
 'Steakhouse',
 'Sushi Restaurant',
 'Taco Place',
 'Tennis Court',
 'Thrift / Vintage Store',
 'Vegetarian / Vegan Restaurant',
 'Veterinarian',
 'Win

I have many duplicate values. I can drop the extra values with the 'drop_duplicates' function. 
This function won´t drop only categories, it will drop the duplicate rows.  I will call it 'TN_VenuesU', the new Data Frame.


In [31]:
TN_VenuesU= TN_Venues.drop_duplicates()
TN_VenuesU.shape# Here is the new shape

(161, 5)

I didn´t consider the 'likes','dislikes' and 'tipCount' features because in the region is very uncommon  post on facebook the local places that you have visited. But I will get it anyway. Maybe I can use them later. By now, the rating feature is very important. I will use the 'details' endpoint again. The stats is a very important feature, but in this JSON file
there is not the 'Checkin' key, so the stats feature is not very important here. 

In [32]:
likes = []
dislikes = []
TipCounts = []
Rating = []
for k in range(len(TN_VenuesU)):
    
    venue_id = '{}'.format(TN_VenuesU['ID'][k]) # ID 
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    url
    result = requests.get(url).json()
    
    likes.append(result['response']['venue']['likes'])
    dislikes.append(result['response']['venue']['dislike'])
    TipCounts.append(result['response']['venue']['stats']['tipCount'])
    Rating.append(result['response']['venue']['rating'])
    
print(len(likes))
print(len(dislikes))
print(len(TipCounts))
print(len(Rating))



KeyError: 'venue'

I need to add the new lists to my Data Frame. 

In [ ]:
TN_VenuesU['likes'] = likes# You can add a new column from a list with this code.
TN_VenuesU['dislikes'] = dislikes# You can add a new column from a list with this code.
TN_VenuesU['Tip_Counts'] = TipCounts# You can add a new column from a list with this code.
TN_VenuesU['Rating'] = Rating# You can add a new column from a list with this code.
TN_VenuesU.head()

In [ ]:
likes[0]['count']#With this code you can extract all the keys that you forgot. 

In [ ]:
Likes = []
for i in range(len(likes)):
    Likes.append(likes[i]['count'])
    
Likes
    

In [ ]:
TN_VenuesU.drop('likes',axis=1,inplace = True)# I drop the old column
TN_VenuesU['likes'] = Likes # I add the new column
TN_VenuesU.head()

I saved the last data frame like 'TN_VenuesUC' and then I imported it. 

In [33]:

body = client_30e6154c81ce458e9a81cf92d1db1713.get_object(Bucket='courseracapstone-donotdelete-pr-tjqo81z1sickby',Key='TN_VenuesUC.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

TN_VenuesUC = pd.read_csv(body)
TN_VenuesUC.head()


,ID,Name,Main Categorie,Latitude,Longitude,dislikes,Tip_Counts,Rating,likes
0,4cae778aeb65b1f760595ccd,Los Taquitos,Taco Place,19.295809,-99.055430,False,37,8.8,102
1,4eaadaf4b803cf1ffb47a0d8,Carnitas La Flor de Cotija,Mexican Restaurant,19.294897,-99.057706,False,13,8.5,32
2,50521cfbe4b02dee62ca3635,Carnitas Estilo Michoacán El Paisa,Taco Place,19.287660,-99.050957,False,10,8.2,11
3,4f20c6d4e4b08ed6b4ca47a2,Don Cuco,Taco Place,19.291856,-99.055175,False,24,8.1,68
4,4f6fe82ee4b07e41f09878a5,Sushisimo,Sushi Restaurant,19.297009,-99.054334,False,16,8.0,35


I will create the 'A' Data Frame with 'Main Categorie' and 'Rating' feature. I´m going to build the Data Frame with the parameters that I want to do in my analysis. 

In [34]:
A = TN_VenuesUC[['Main Categorie','Rating']]# You can create a new Data Frame with less features. 
B = A.groupby(by= 'Main Categorie').mean()# This line of code allows you to get the rating average of each categorie. 
B


,Rating
Main Categorie,
American Restaurant,7.600000
Art Museum,8.900000
Asian Restaurant,7.700000
BBQ Joint,8.066667
Bakery,8.100000
Bar,7.850000
Beer Garden,7.850000
Breakfast Spot,8.175000
Brewery,7.800000


In [35]:
A['1'] = 1 # You can add a numercical column with this line of code. 
B1 = A[['Main Categorie','1']] # the B Data Frame in order to count the categories. 

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [36]:
B1.head()

,Main Categorie,1
0,Taco Place,1
1,Mexican Restaurant,1
2,Taco Place,1
3,Taco Place,1
4,Sushi Restaurant,1


In [37]:
B2 = B1.groupby(by = 'Main Categorie').sum()# With this line of code you can Sum the values with the same feature, in this case 'Main Categorie'
B2

,1
Main Categorie,
American Restaurant,1
Art Museum,1
Asian Restaurant,1
BBQ Joint,3
Bakery,4
Bar,2
Beer Garden,2
Breakfast Spot,4
Brewery,1


I already created the Data frames with the numerical features. I need to merge them.

In [38]:
B3 = pd.merge(B,B2, on = 'Main Categorie')# With this line of code you can merge Data Frames with one column in common. 
B3.head()

,Rating,1
Main Categorie,,
American Restaurant,7.600000,1
Art Museum,8.900000,1
Asian Restaurant,7.700000,1
BBQ Joint,8.066667,3
Bakery,8.100000,4


### The most popular venues in the Cuahtémoc borough sorted by rating.

In [43]:
# I opened the CN Data Frame
body = client_30e6154c81ce458e9a81cf92d1db1713.get_object(Bucket='courseracapstone-donotdelete-pr-tjqo81z1sickby',Key='CNV_byRating.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

CNV_byRating = pd.read_csv(body)
CNV_byRating.head(18)


,index,ID,Name,Rating,Main Categorie
0,90,4baec3d5f964a52018d63be3,Nevería Roxy,9.4,Ice Cream Shop
1,50,4b684e6ef964a520d0702be3,Museo de Arte Popular,9.4,Art Museum
2,117,4b819a4cf964a520fcb130e3,Fondo de Cultura Económica Rosario Castellanos,9.3,Bookstore
3,55,50ca02bf245f2d4aa8c2aeef,El Cardenal,9.3,Mexican Restaurant
4,129,51ba6ba9498e9bcd16eee9ec,Cachito Mío Quiches & Tartas,9.2,Coffee Shop
5,120,56d76bac498ecdce6449901f,Tierra Garat,9.1,Cafeteria
6,140,4ccaca619b9346889cb65041,Kiosco Morisco,9.1,Plaza
7,80,4d49c47b11a36ea827282b1c,Zéfiro,9.1,Mexican Restaurant
8,10,55f476fe498e26410387a491,Hotel Historico Central,9.1,Hotel
9,35,4bfc1626d6f2c9b669b24ec8,Antiguo Colegio de San Ildefonso,9.0,Art Museum


### The most popular venues in the Tláhuac borough sorted by Rating.

Here is the Data Frame sorted by Rating

In [40]:
B3.sort_values('Rating',ascending=False) 

,Rating,1
Main Categorie,,
Pet Store,9.200000,1
Sports Club,9.000000,1
Art Museum,8.900000,1
College Gym,8.800000,1
Market,8.800000,1
Residential Building (Apartment / Condo),8.700000,1
Veterinarian,8.700000,1
Steakhouse,8.600000,1
Pool,8.600000,1


### The most popular venues in the Tláhuac borough sorted by number of venues.

Here is the Data Frame sorted by number of venues

In [42]:
B3.sort_values('1',ascending=False)

,Rating,1
Main Categorie,,
Mexican Restaurant,7.957143,28
Taco Place,7.961905,21
Seafood Restaurant,7.655556,9
Pizza Place,7.800000,6
Ice Cream Shop,8.233333,6
Coffee Shop,7.966667,6
Gym / Fitness Center,7.620000,5
Gym,7.520000,5
Breakfast Spot,8.175000,4


### That´s it for now. Thank you!